In [1]:
import time
import numpy as np
import xarray as xr
from pathlib import Path

from pyPulses.utils import SweepMeasureProduct
from ez_data import DsetConnector, sqlite_to_xarray

In [2]:
VG_ = 0.0
def VG(x = None):
    global VG_
    if x is None: 
        return VG_
    VG_ = x

VB_ = 0.0
def VB(x = None):
    global VB_
    if x is None: 
        return VB_
    VB_ = x

def I():
    return np.random.normal(loc = 1.0, scale = 0.001)

def V():
    return np.random.normal(loc = 0.0, scale = 0.01)

In [3]:
# Performance without any data saving

sweep = SweepMeasureProduct(
    coordinates    = [{'f': VG, 'name': 'VG', 'long_name': R"$V_G$", 'units': 'V'},
                      {'f': VB, 'name': 'VB', 'long_name': R"$V_B$", 'units': 'V'}],
    measurements   = [{'f': I, 'name': 'I', 'long_name': R"$I$", 'units': 'A'},
                      {'f': V, 'name': 'V', 'long_name': R"$V$", 'units': 'V'}],
    time_per_point = 0.01,
    axes           = (np.linspace(-1, 1, 50), np.linspace(-0.5, 0.5, 40)),
    retain_return  = False,
    plot_fields    = None,
    timestamp      = True
)

t0 = time.perf_counter()
sweep.run()
t1 = time.perf_counter()

print(f"Sweep finished in {t1 - t0:.3f} seconds")

Sweep finished in 20.835 seconds


In [4]:
# Performance using DsetConnector

sweep = SweepMeasureProduct(
    coordinates    = [{'f': VG, 'name': 'VG', 'long_name': R"$V_G$", 'units': 'V'},
                      {'f': VB, 'name': 'VB', 'long_name': R"$V_B$", 'units': 'V'}],
    measurements   = [{'f': I, 'name': 'I', 'long_name': R"$I$", 'units': 'A'},
                      {'f': V, 'name': 'V', 'long_name': R"$V$", 'units': 'V'}],
    time_per_point = 0.01,
    axes           = (np.linspace(-1, 1, 50), np.linspace(-0.5, 0.5, 40)),
    retain_return  = False,
    plot_fields    = None,
    timestamp      = True
)

demo_path = Path("demo/demo_sweep")
connector = DsetConnector.from_param_sweep(sweep, demo_path)

sweep.post_callback = connector.add_point
t0 = time.perf_counter()
sweep.run()
connector.close()
t1 = time.perf_counter()

print(f"Sweep finished in {t1 - t0:.3f} seconds")

Sweep finished in 21.261 seconds


In [5]:
# Performance using DsetConnector with buffering

sweep = SweepMeasureProduct(
    coordinates    = [{'f': VG, 'name': 'VG', 'long_name': R"$V_G$", 'units': 'V'},
                      {'f': VB, 'name': 'VB', 'long_name': R"$V_B$", 'units': 'V'}],
    measurements   = [{'f': I, 'name': 'I', 'long_name': R"$I$", 'units': 'A'},
                      {'f': V, 'name': 'V', 'long_name': R"$V$", 'units': 'V'}],
    time_per_point = 0.01,
    axes           = (np.linspace(-1, 1, 50), np.linspace(-0.5, 0.5, 40)),
    retain_return  = False,
    plot_fields    = None,
    timestamp      = True
)

demo_path = Path("demo/demo_sweep_buffered")
connector = DsetConnector.from_param_sweep(sweep, demo_path, use_buffer = True)

sweep.post_callback = connector.add_point
t0 = time.perf_counter()
sweep.run()
connector.close()
t1 = time.perf_counter()

print(f"Sweep finished in {t1 - t0:.3f} seconds")

Sweep finished in 20.880 seconds


In [6]:
ds1 = sqlite_to_xarray("demo/demo_sweep")
ds2 = sqlite_to_xarray("demo/demo_sweep_buffered")
display(ds1)
display(ds2)

<xarray.Dataset> Size: 81kB
Dimensions:    (dim0: 50, dim1: 40)
Coordinates:
    VG         (dim0, dim1) float64 16kB -1.0 -1.0 -1.0 -1.0 ... 1.0 1.0 1.0 1.0
    VB         (dim0, dim1) float64 16kB -0.5 -0.4744 -0.4487 ... 0.4744 0.5
  * dim0       (dim0) int64 400B 0 1 2 3 4 5 6 7 8 ... 42 43 44 45 46 47 48 49
  * dim1       (dim1) int64 320B 0 1 2 3 4 5 6 7 8 ... 32 33 34 35 36 37 38 39
Data variables:
    I          (dim0, dim1) float64 16kB 0.9998 1.001 1.0 1.0 ... 1.0 1.0 0.9993
    V          (dim0, dim1) float64 16kB -0.005638 0.01609 ... 0.001385 0.001973
    timestamp  (dim0, dim1) object 16kB '2025-09-11 08:08:49.835366' ... '202...
Attributes:
    time_per_point:  0.01

<xarray.Dataset> Size: 81kB
Dimensions:    (dim0: 50, dim1: 40)
Coordinates:
    VG         (dim0, dim1) float64 16kB -1.0 -1.0 -1.0 -1.0 ... 1.0 1.0 1.0 1.0
    VB         (dim0, dim1) float64 16kB -0.5 -0.4744 -0.4487 ... 0.4744 0.5
  * dim0       (dim0) int64 400B 0 1 2 3 4 5 6 7 8 ... 42 43 44 45 46 47 48 49
  * dim1       (dim1) int64 320B 0 1 2 3 4 5 6 7 8 ... 32 33 34 35 36 37 38 39
Data variables:
    I          (dim0, dim1) float64 16kB 0.9993 1.001 1.002 ... 0.9996 0.9993
    V          (dim0, dim1) float64 16kB 0.003003 0.006856 ... -0.00542 0.01077
    timestamp  (dim0, dim1) object 16kB '2025-09-11 08:09:11.116977' ... '202...
Attributes:
    time_per_point:  0.01

In [2]:
ds1 = sqlite_to_xarray("demo/demo_sweep", 'stack')
ds2 = sqlite_to_xarray("demo/demo_sweep_buffered", 'stack')
display(ds1)
display(ds2)

<xarray.Dataset> Size: 161kB
Dimensions:    (dim0: 50, dim1: 40, repeat: 2)
Coordinates:
    VG         (dim0, dim1, repeat) float64 32kB -1.0 -1.0 -1.0 ... 1.0 1.0 1.0
    VB         (dim0, dim1, repeat) float64 32kB -0.5 -0.5 -0.4744 ... 0.5 0.5
  * repeat     (repeat) int64 16B 0 1
  * dim0       (dim0) int64 400B 0 1 2 3 4 5 6 7 8 ... 42 43 44 45 46 47 48 49
  * dim1       (dim1) int64 320B 0 1 2 3 4 5 6 7 8 ... 32 33 34 35 36 37 38 39
Data variables:
    I          (dim0, dim1, repeat) float64 32kB 0.9995 1.0 ... 0.9996 0.9991
    V          (dim0, dim1, repeat) float64 32kB -0.02141 0.01013 ... -0.002186
    timestamp  (dim0, dim1, repeat) object 32kB '2025-09-10 17:32:56.919668' ...
Attributes:
    time_per_point:  0.01

<xarray.Dataset> Size: 161kB
Dimensions:    (dim0: 50, dim1: 40, repeat: 2)
Coordinates:
    VG         (dim0, dim1, repeat) float64 32kB -1.0 -1.0 -1.0 ... 1.0 1.0 1.0
    VB         (dim0, dim1, repeat) float64 32kB -0.5 -0.5 -0.4744 ... 0.5 0.5
  * repeat     (repeat) int64 16B 0 1
  * dim0       (dim0) int64 400B 0 1 2 3 4 5 6 7 8 ... 42 43 44 45 46 47 48 49
  * dim1       (dim1) int64 320B 0 1 2 3 4 5 6 7 8 ... 32 33 34 35 36 37 38 39
Data variables:
    I          (dim0, dim1, repeat) float64 32kB 0.9999 0.9988 ... 0.9986 1.0
    V          (dim0, dim1, repeat) float64 32kB 0.002652 0.003353 ... 0.007493
    timestamp  (dim0, dim1, repeat) object 32kB '2025-09-10 17:33:18.193599' ...
Attributes:
    time_per_point:  0.01